In [1]:
pip install colormaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib import colors as mcolors
import colormaps as cmaps

import xarray as xr
import numpy as np
from numba import njit

In [3]:
path = '/home/stratum2/ruisdael/bart/corso/vinh_tan/outer'

u = xr.open_dataset(f'{path}/u.xy.nc', decode_times=False)
v = xr.open_dataset(f'{path}/v.xy.nc', decode_times=False)
no2 = xr.open_dataset(f'{path}/no2_path.xy.nc', decode_times=False)

u = u.isel(z=1)
v = v.isel(z=1)

xsize = 172800
ysize = 172800

itot = 576
jtot = 576

# Assume u,v are at same location (cheating!).
dx = xsize / itot
dy = ysize / jtot

x = np.arange(dx/2, xsize, dx)
y = np.arange(dy/2, ysize, dy)

In [6]:
@njit
def integrate_kernel(xp, yp, index, alpha, start_step, step, u, v, dx, dy, dt, extent):
    """
    Integrate barticles to next location.
    """
    max_length = xp[0,:].size

    start_taper = int(0.5 * max_length)
    length_taper = max_length - start_taper

    for n in range(index.size):
        if step < start_step[n]:
            continue

        x = xp[n, index[n]]
        y = yp[n, index[n]]

        # Check if next time is out of bounds. If so, reset particle.
        if index[n] >= max_length-1 or x < extent[0] or x > extent[1] or y < extent[2] or y > extent[3]:

            xp[n, :] = 0
            yp[n, :] = 0
            index[n] = 0
            alpha[n,:] = 0

            xp[n, 0] = np.random.uniform(extent[0], extent[1])
            yp[n, 0] = np.random.uniform(extent[2], extent[3])

            # Start is a bit weird; we can't draw a line segment from only one point..
            xp[n, 1] = xp[n, 0]
            yp[n, 1] = yp[n, 0]

        # Read coordinates again; possibly reset by .. reset.
        x = xp[n, index[n]]
        y = yp[n, index[n]]

        # Fractional index.
        i_f = (x-(dx/2))/dx
        j_f = (y-(dy/2))/dy

        # Left index.
        i_l = int(i_f)
        j_l = int(j_f)

        # Offset from left index.
        di = i_f - i_l
        dj = j_f - j_l

        # Bi-linear interpolation to barticle location.
        u_i = (1-dj) * (1-di) * u[j_l,   i_l  ] + \
                 dj  * (1-di) * u[j_l+1, i_l  ] + \
              (1-dj) *    di  * u[j_l,   i_l+1] + \
                 dj  *    di  * u[j_l+1, i_l+1]

        v_i = (1-dj) * (1-di) * v[j_l,   i_l  ] + \
                 dj  * (1-di) * v[j_l+1, i_l  ] + \
              (1-dj) *    di  * v[j_l,   i_l+1] + \
                 dj  *    di  * v[j_l+1, i_l+1]

        xp[n, index[n]+1] = x + u_i * dt
        yp[n, index[n]+1] = y + v_i * dt
        
        # Line segment from `i` to `i+1` using `alpha[i]`..
        # Last steps of integration the stream "dissapears" (magic!).
        if index[n] < start_taper:
            alpha[n, index[n]] = 1.0
        else:
            a = max(0.0, 1.0 - (index[n] - start_taper) / (length_taper - 1))
            alpha[n, :index[n]+1] = a

        index[n] += 1



def repeated_sequence(N, repeat_count):
    num_full_groups = N // repeat_count
    remainder = N % repeat_count
    arr = np.repeat(np.arange(num_full_groups), repeat_count)
    if remainder > 0:
        arr = np.concatenate([arr, np.full(remainder, num_full_groups)])
    return arr


class Streamlines:
    def __init__(self, x, y, n_streams, max_length, dt, float_type=np.float32):
        """
        Class to create Windy-style streamline for plotting with e.g. Matplotlib.

        Parameters:
        ----------
        x : np.ndarray(float, ndim=1)
            x-location of grid.
        y : np.ndarray(float, ndim=1)
            y-location of grid.
        n_streams : int
            Number of streamlines.
        max_length : int
            Max length of steamline.
        dt : float
            Integration time step.

        Methods:
        -------
        step(u, v)
            Give `u,v` field, advance all particles on time step.
        reset(index):
            Reset barticle/streamline to zero length and no opacity.
        """

        self.x = x
        self.y = y

        self.n_streams = n_streams
        self.max_length = max_length
        self.dt = dt
        self.float_type = float_type

        # Bounds of barticles.
        self.x_min = x[0]
        self.x_max = x[-1]
        self.y_min = y[0]
        self.y_max = y[-1]

        self.extent = [self.x_min, self.x_max, self.y_min, self.y_max]

        self.dx = x[1] - x[0]
        self.dy = y[1] - y[0]

        # Data arrays.
        self.x_stream = np.zeros((self.n_streams, self.max_length), dtype=float_type)   # x-coordinates streamline.
        self.y_stream = np.zeros((self.n_streams, self.max_length), dtype=float_type)   # y-coordinates streamline.
        self.alpha    = np.zeros((self.n_streams, self.max_length), dtype=float_type)   # Opacity streamline.
        self.index    = np.zeros(self.n_streams, dtype=np.uint16)                       # Age of particles in # steps.

        #self.start_step = np.arange(self.n_streams)                                     # Delayed start of individual streams.
        self.start_step = repeated_sequence(self.n_streams, 10) # Start in gropus.

        # Random init of particle location.
        self.x_stream[:, 0] = np.random.uniform(self.x_min, self.x_max, size=self.n_streams)
        self.y_stream[:, 0] = np.random.uniform(self.y_min, self.y_max, size=self.n_streams)
        self.alpha[:,0] = 0
        self.index[:] = 0

        self.step = 0


    def integrate(self, u, v):
        """
        Advance all streamline one time step.

        Parameters:
        u : np.ndarray(float, ndim=2)
            u-velocity component.
        v : np.ndarray(float, ndim=2)
            v-velocity component.
        """

        integrate_kernel(self.x_stream, self.y_stream, self.index, self.alpha, self.start_step, self.step, u, v, self.dx, self.dy, self.dt, self.extent)
        self.step += 1


    def plot(self, ax, color='k', linewidth=1, alpha=1):
        """
        Plot streamlines on `ax`.
        """

        points = np.stack([self.x_stream, self.y_stream], axis=-1)
        segments = np.stack([points[:, :-1], points[:, 1:]], axis=2)
        segments = segments.reshape(-1, 2, 2)
        alpha_segments = self.alpha[:, :-1].ravel() * alpha

        base_color = np.array(mcolors.to_rgba(color), dtype=np.float32)
        colors = np.tile(base_color, (alpha_segments.size, 1))
        colors[:, 3] = alpha_segments

        lc = LineCollection(segments, linewidth=linewidth, colors=colors, linewidths=1)
        ax.add_collection(lc)
        ax.autoscale_view()


stream = Streamlines(x, y, n_streams=1000, max_length=50, dt=30.0)

t0 = 1000
t1 = 1500

norm = mcolors.LogNorm(vmin=2e-7, vmax=1e-5)

for i in range(t0, t1):
    print(i)

    stream.integrate(u.u[i,:,:].values, v.v[i,:,:].values)

    plt.figure(figsize=(8, 6), layout='constrained')
    ax=plt.gca()
    plt.pcolormesh(x, y, no2.no2_path[i,:,:], cmap=cmaps.WhiteBlueGreenYellowRed, norm=norm)
    stream.plot(ax, color='k', linewidth=1, alpha=0.5)
    plt.xlim(0, xsize)
    plt.ylim(0, ysize)
    plt.savefig(f'figs/fig_{i-t0:05d}.png')
    plt.close('all')

1000
1001
1002
1003
1004
1005
1006
1007
1008
1009
1010
1011
1012
1013
1014
1015
1016
1017
1018
1019
1020
1021
1022
1023
1024
1025
1026
1027
1028
1029
1030
1031
1032
1033
1034
1035
1036
1037
1038
1039
1040
1041
1042
1043
1044
1045
1046
1047
1048
1049
1050
1051
1052
1053
1054
1055
1056
1057
1058
1059
1060
1061
1062
1063
1064
1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131
1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164
1165
1166
1167
1168
1169
1170
1171
1172
1173
1174
1175
1176
1177
1178
1179
1180
1181
1182
1183
1184
1185
1186
1187
1188
1189
1190
1191
1192
1193
1194
1195
1196
1197
1198
1199
